In [7]:
!pip install gender-guesser

In [6]:
import gender_guesser.detector as gender
import pandas as pd

In [74]:
# Collect first names
df = pd.read_csv("all_sorted_collection.csv")

first_names = []
for index, row in df.iterrows():
    if row['Coll_head'] == 1:
        # if name contains a comma, it's likely a name and not an organization
        if ", " in str(row['Name']):
            name = str(row['Name'].split(" ")[1])
            # Find first name after Sir or Mrs.
            if ("Sir" in name or "Mrs" in name or "Dr." in name or "Capt" in name or "Miss" in name or "Lord" in name or "Doctor" in name) and len(row['Name'].split(" ")) > 2:
                name = str(row['Name'].split(" ")[2])
            # For initials, try to find whole name
            if len(name) < 2 and len(row['Name'].split(" ")) > 2 and len(str(row['Name'].split(" ")[2])) > 2:
                name = str(row['Name'].split(" ")[2])
            if not "Archive" in name and not "Army" in name and not "University" in name and not "Family" in name and not "Ministry" in name and not "CHURCH" in name.upper():
                name = name.strip(" \r\n,().")
                if not "." in name and len(name) > 3:
                    first_names.append(name)

In [76]:
# Detect gender for each main entry author
d = gender.Detector()

gender_counts = {
    "unknown": 0,
    "andy": 0,
    "male": 0,
    "female": 0,
    "mostly_male": 0,
    "mostly_female": 0
}

# Determine gender of each name
for name in first_names:
    gen = d.get_gender(name)
    if gen == "unknown":
        # Common unclassified name
        if name == "Duff":
            gen = "male"
        # Check for trailing e's (common in names from OCR error) and recheck those names
        elif "e" == name[len(name)-1]:
            name = name[0:len(name)-1]
            gen = d.get_gender(name)
    gender_counts[gen] += 1
    
print(gender_counts)

{'unknown': 980, 'andy': 18, 'male': 9520, 'female': 932, 'mostly_male': 67, 'mostly_female': 149}
